Examples of taking Jacobians, indexing into them according joints, etc.

In [1]:
import numpy as np

from pydrake.common import FindResourceOrThrow
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.tree import JacobianWrtVariable, JointIndex

In [2]:
np.set_printoptions(formatter={"float_kind": lambda x: f"{x:.3f}"})

In [3]:
def get_joints(plant, model_instance):
    joints = []
    for i in range(plant.num_joints()):
        joint = plant.get_joint(JointIndex(i))
        if joint.model_instance() == model_instance:
            joints.append(joint)
    return joints

In [4]:
def as_lines(items):
    return "\n".join(str(x) for x in items)

In [5]:
plant = MultibodyPlant(time_step=0.0)

parser = Parser(plant)
iiwa_file = FindResourceOrThrow(
   "drake/manipulation/models/iiwa_description/sdf/"
   "iiwa14_no_collision.sdf")

iiwa_1 = parser.AddModelFromFile(iiwa_file, model_name="iiwa_1")
plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=plant.GetFrameByName("iiwa_link_0", iiwa_1),
    X_PC=RigidTransform([-3, 0 ,0]),
)

iiwa_2 = parser.AddModelFromFile(iiwa_file, model_name="iiwa_2")
plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=plant.GetFrameByName("iiwa_link_0", iiwa_2),
    X_PC=RigidTransform([3, 0 ,0]),
)

plant.Finalize()
context = plant.CreateDefaultContext()

In [6]:
joints_1 = get_joints(plant, iiwa_1)
joints_2 = get_joints(plant, iiwa_2)

print(as_lines(joints_1))
print()
print(as_lines(joints_2))

<RevoluteJoint_[float] name='iiwa_joint_1' index=0 model_instance=2>
<RevoluteJoint_[float] name='iiwa_joint_2' index=1 model_instance=2>
<RevoluteJoint_[float] name='iiwa_joint_3' index=2 model_instance=2>
<RevoluteJoint_[float] name='iiwa_joint_4' index=3 model_instance=2>
<RevoluteJoint_[float] name='iiwa_joint_5' index=4 model_instance=2>
<RevoluteJoint_[float] name='iiwa_joint_6' index=5 model_instance=2>
<RevoluteJoint_[float] name='iiwa_joint_7' index=6 model_instance=2>
<WeldJoint_[float] name='WorldBody_welds_to_iiwa_link_0' index=7 model_instance=2>

<RevoluteJoint_[float] name='iiwa_joint_1' index=8 model_instance=3>
<RevoluteJoint_[float] name='iiwa_joint_2' index=9 model_instance=3>
<RevoluteJoint_[float] name='iiwa_joint_3' index=10 model_instance=3>
<RevoluteJoint_[float] name='iiwa_joint_4' index=11 model_instance=3>
<RevoluteJoint_[float] name='iiwa_joint_5' index=12 model_instance=3>
<RevoluteJoint_[float] name='iiwa_joint_6' index=13 model_instance=3>
<RevoluteJoint_

In [7]:
def get_velocity_mask(plant, joints):
    """
    Generates a mask according to supplied set of ``joints``.

    The binary mask is unable to preserve ordering for joint indices, thus
    `joints` required to be a ``set`` (for simplicity).
    """
    assert isinstance(joints, set)
    mask = np.zeros(plant.num_velocities(), dtype=np.bool)
    for joint in joints:
        start = joint.velocity_start()
        end = start + joint.num_velocities()
        mask[start:end] = True
    return mask

def get_velocity_indices(plant, joints):
    """
    Generates a list of indices according to supplies list of ``joints``.

    The indices are generated according to the order of ``joints``, thus
    ``joints`` is required to be a list (for simplicity).
    """
    indices = []
    for joint in joints:
        start = joint.velocity_start()
        end = start + joint.num_velocities()
        for i in range(start, end):
            indices.append(i)
    return indices

In [8]:
# Option 1. Ditch order.
mask_1 = get_velocity_mask(plant, set(joints_1))
mask_2 = get_velocity_mask(plant, set(joints_2))

print(mask_1.astype(np.uint8))
print(mask_2.astype(np.uint8))

[1 0 1 0 1 0 1 0 1 0 1 0 1 0]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1]


In [9]:
# Option 2. This preserves order.
indices_1 = get_velocity_indices(plant, joints_1)
indices_2 = get_velocity_indices(plant, joints_2)

print(indices_1)
print(indices_2)

print()

# Show order sensitivity.
print(get_velocity_indices(plant, joints_1[::-1]))
print(get_velocity_indices(plant, joints_2[::-1]))

[0, 2, 4, 6, 8, 10, 12]
[1, 3, 5, 7, 9, 11, 13]

[12, 10, 8, 6, 4, 2, 0]
[13, 11, 9, 7, 5, 3, 1]


In [10]:
def calc_velocity_jacobian(plant, context, frame_A, frame_B):
    Jv_AB = plant.CalcJacobianSpatialVelocity(
        context,
        with_respect_to=JacobianWrtVariable.kV,
        frame_B=frame_B,
        p_BP=[0, 0, 0],
        frame_A=frame_A,
        frame_E=frame_A,
    )
    return Jv_AB

In [11]:
# Posture the arms such that each Jacobian (w.r.t. each final link) is relatively unique.
plant.SetPositions(context, iiwa_1, np.deg2rad([45, 30, 0, -30, 45, 45, 45]))
plant.SetPositions(context, iiwa_2, np.deg2rad([-60, -15, 0, 90, 30, 15, 0]))

In [12]:
frame_W = plant.world_frame()
frame_G1 = plant.GetFrameByName("iiwa_link_7", iiwa_1)
frame_G2 = plant.GetFrameByName("iiwa_link_7", iiwa_2)

Jv_WG1 = calc_velocity_jacobian(plant, context, frame_W, frame_G1)
Jv1_WG1 = Jv_WG1[:, mask_1]
Jv_WG2 = calc_velocity_jacobian(plant, context, frame_W, frame_G2)
Jv2_WG2 = Jv_WG2[:, mask_2]

print(Jv1_WG1)
print()
print(Jv2_WG2)

[[0.000 -0.707 0.354 0.707 0.612 -0.750 0.256]
 [0.000 0.707 0.354 -0.707 0.612 0.250 0.963]
 [1.000 -0.000 0.866 -0.000 0.500 0.612 -0.079]
 [-0.471 0.394 -0.211 -0.137 -0.043 -0.049 0.000]
 [0.414 0.394 0.162 -0.137 0.014 0.008 0.000]
 [0.000 -0.626 0.020 0.416 0.035 -0.064 0.000]]

[[0.000 0.866 -0.129 -0.866 -0.483 0.815 -0.383]
 [0.000 0.500 0.224 -0.500 0.837 0.321 0.923]
 [1.000 -0.000 0.966 -0.000 -0.259 -0.483 -0.033]
 [-0.504 0.150 -0.419 0.053 0.017 0.035 0.000]
 [-0.279 -0.259 -0.230 -0.092 0.007 0.017 0.000]
 [0.000 0.575 -0.003 -0.467 -0.010 0.071 0.000]]
